In [1]:
from datasets import load_dataset
from random import randrange
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,TrainingArguments,pipeline
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer

e:\code\insight_text_game\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os

current_dir = os.path.dirname(os.path.realpath(__file__))
gemma_model_dir = os.path.join(current_dir, "../gemma_model")

In [2]:

#train & test.json are in same folder as the jupyter notebook
data_files = {'train':'train.json','test':'test.json'}
dataset = load_dataset('json',data_files=data_files)

model = AutoModelForSeq2SeqLM.from_pretrained(gemma_model_dir)

#Makes training faster but a little less accurate 
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(gemma_model_dir, trust_remote_code=True)

#setting padding instructions for tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

FileNotFoundError: Unable to find 'E:/code/insight_text_game/fine_tuning\train.json'